In [1]:
import graphlab

In [2]:
import psycopg2

This non-commercial license of GraphLab Create for academic use is assigned to mccoy@nyu.edu and will expire on January 19, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1485036630.log


In [3]:
conn = psycopg2.connect('dbname=osp')

In [4]:
import sqlite3

labels_conn = sqlite3.connect('/home/sam/labeler/labels.sqlite')

In [5]:
labels = graphlab.SFrame.from_sql(labels_conn, 'SELECT document_id, label FROM labels')

In [17]:
syllabi = graphlab.SFrame.from_sql(conn, 'SELECT document_id, text FROM document_text WHERE document_id IN ({})'.format(','.join(labeled_ids)))

In [16]:
labeled_ids = [str(x.item()) for x in labels['document_id'].to_numpy()]

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('rf', RandomForestClassifier(n_estimators=100))
])

from sklearn.model_selection import cross_val_score

In [ ]:
data = labels.join(syllabi, on='document_id', how='left')

In [30]:
data['is_syllabus'] = data['label'] != 0

In [34]:
scores = cross_val_score(text_clf, data['text'].to_numpy(), data['is_syllabus'].to_numpy(), cv=5)

scores

array([ 0.93478261,  0.76086957,  0.86666667,  0.86363636,  0.72727273])

In [39]:
import scipy

scipy.stats.describe(scores)

DescribeResult(nobs=5, minmax=(0.72727272727272729, 0.93478260869565222), mean=0.8306455862977602, variance=0.0071962675387662496, skewness=-0.10640433167880783, kurtosis=-1.4409150243063844)
